In [ ]:
import pytest
import json
from unittest.mock import Mock, patch, AsyncMock, MagicMock
import datetime as dt
from freezegun import freeze_time

# Import the modules to test
from ingest import (
    backfill_ingestion_earnings,
    daily_ingestion_earnings,
    get_s3_credentials,
    pull_ingest_update
)
from vespa_manager import VespaManager

# Test data fixtures
@pytest.fixture
def sample_payload():
    return {
        "nativeId": "test123",
        "fields": {
            "tickers_s": ["AAPL", "GOOGL"],
            "year_s": "2025",
            "quarter_s": "Q1",
            "event_time_s": "2025-01-15T10:00:00Z"
        }
    }

@pytest.fixture
def vespa_manager():
    return VespaManager(
        schema_id="test_schema",
        env="test_env",
        s3_bucket_id="test_bucket",
        creds={"access_key": "test_key", "secret_key": "test_secret"},
        logging_dir="test_logs"
    )

# Tests for ingest.py
class TestIngest:
    @patch('ingest.asyncio.run')
    @patch('ingest.my_dict')
    def test_backfill_ingestion_earnings(self, mock_my_dict, mock_asyncio_run):
        # Setup
        mock_my_dict.items.return_value = [
            ("test_dir", ["2025-01-01", "2025-01-02"])
        ]
        
        # Execute
        backfill_ingestion_earnings("test_schema", "test_bucket")
        
        # Assert
        mock_asyncio_run.assert_called()
        assert mock_asyncio_run.call_count == 1

    @freeze_time("2025-01-31")
    @patch('ingest.asyncio.run')
    @patch('ingest.job_date_if_none')
    def test_daily_ingestion_earnings(self, mock_job_date, mock_asyncio_run):
        # Setup
        mock_job_date.return_value = "20250131"
        
        # Execute
        daily_ingestion_earnings("test_schema", "test_bucket")
        
        # Assert
        mock_asyncio_run.assert_called_once()
        assert "2025_ECT/20250131" in str(mock_asyncio_run.call_args)

    @patch('ingest.requests.get')
    def test_get_s3_credentials(self, mock_get):
        # Setup
        mock_response = Mock()
        mock_response.json.return_value = {
            "data": {"data": {"key": "value"}}
        }
        mock_get.return_value = mock_response
        
        # Execute
        result = get_s3_credentials()
        
        # Assert
        assert result == {"key": "value"}
        mock_get.assert_called_once()

    @pytest.mark.asyncio
    async def test_pull_ingest_update(self):
        with patch('ingest.get_s3_credentials') as mock_creds, \
             patch('ingest.VespaManager') as mock_vespa_manager, \
             patch('ingest.load_earnings_call_data_vespa') as mock_load_data, \
             patch('ingest.get_payload') as mock_get_payload:
            
            # Setup
            mock_creds.return_value = {"key": "value"}
            mock_manager = AsyncMock()
            mock_manager.get_failed_payloads.return_value = {}
            mock_vespa_manager.return_value = mock_manager
            mock_load_data.return_value = "test_df"
            mock_get_payload.return_value = ["test_payload"]
            
            # Execute
            await pull_ingest_update(
                start_date=dt.date(2025, 1, 1),
                end_date=dt.date(2025, 1, 31),
                schema_id="test_schema",
                env="test_env",
                s3_bucket_id="test_bucket",
                logging_dir="test_logs"
            )
            
            # Assert
            mock_manager.ingest_in_vespa.assert_called_once_with(["test_payload"])
            mock_manager.update_ingestion_status.assert_called()

# Tests for vespa_manager.py
class TestVespaManager:
    @pytest.mark.asyncio
    async def test_ingest_in_vespa(self, vespa_manager, sample_payload):
        with patch.object(vespa_manager, 'get_ingestion_tracker') as mock_tracker, \
             patch.object(vespa_manager, 'get_ingested_native_id_list') as mock_id_list, \
             patch.object(vespa_manager, 'get_failed_payloads') as mock_failed, \
             patch('vespa_manager.post_with_limit') as mock_post:
            
            # Setup
            mock_tracker.return_value = {}
            mock_id_list.return_value = []
            mock_failed.return_value = {}
            mock_post.return_value = {
                "successDocs": [{
                    "documentId": "doc123"
                }]
            }
            
            # Execute
            await vespa_manager.ingest_in_vespa([sample_payload])
            
            # Assert
            mock_post.assert_called_once()
            assert "doc123" in str(mock_post.call_args)

    def test_update_failed_payloads(self, vespa_manager, sample_payload):
        with patch.object(vespa_manager, 'get_failed_payloads') as mock_failed, \
             patch.object(vespa_manager.s3, 'write_file') as mock_write:
            
            # Setup
            mock_failed.return_value = {}
            
            # Execute
            vespa_manager.update_failed_payloads(
                "test123",
                sample_payload,
                "test error"
            )
            
            # Assert
            mock_write.assert_called_once()
            assert "test error" in str(mock_write.call_args)

    def test_get_failed_payloads(self, vespa_manager):
        with patch.object(vespa_manager.s3, 'file_exists') as mock_exists, \
             patch.object(vespa_manager.s3, 'read_file') as mock_read:
            
            # Setup
            mock_exists.return_value = True
            mock_read.return_value = json.dumps({"test123": {"status": "failed"}})
            
            # Execute
            result = vespa_manager.get_failed_payloads()
            
            # Assert
            assert "test123" in result
            assert result["test123"]["status"] == "failed"

    @pytest.mark.asyncio
    async def test_update_ingestion_status(self, vespa_manager):
        with patch.object(vespa_manager, 'get_ingestion_tracker') as mock_tracker, \
             patch.object(vespa_manager, 'get_failed_payloads') as mock_failed, \
             patch.object(vespa_manager, 'get_ingestion_status') as mock_status, \
             patch.object(vespa_manager.s3, 'read_file') as mock_read, \
             patch.object(vespa_manager.s3, 'file_exists') as mock_exists, \
             patch.object(vespa_manager.s3, 'write_file') as mock_write:
            
            # Setup
            mock_tracker.return_value = {"test123": "doc123"}
            mock_failed.return_value = {}
            mock_status.return_value = "SUCCESS"
            mock_read.return_value = json.dumps({
                "payload": {"nativeId": "test123"}
            })
            mock_exists.return_value = False
            
            # Execute
            failed_payloads, all_processed = await vespa_manager.update_ingestion_status()
            
            # Assert
            assert not failed_payloads
            assert all_processed
            mock_write.assert_called()

    @pytest.mark.asyncio
    async def test_reingest_documents(self, vespa_manager):
        with patch.object(vespa_manager, 'ingest_in_vespa') as mock_ingest:
            # Setup
            failed_docs = [{"nativeId": "test123"}]
            
            # Execute
            await vespa_manager.reingest_documents(failed_docs)
            
            # Assert
            mock_ingest.assert_called_once_with(failed_docs)

    def test_move_to_permanent_failures(self, vespa_manager, sample_payload):
        with patch.object(vespa_manager.s3, 'write_file') as mock_write:
            # Execute
            vespa_manager.move_to_permanent_failures("test123", sample_payload)
            
            # Assert
            mock_write.assert_called_once()
            assert "permanent_failures" in str(mock_write.call_args)

# Additional test configurations
def pytest_configure(config):
    """Optional pytest configuration"""
    config.addinivalue_line(
        "markers",
        "asyncio: mark test as async"
    )